**RELEVANT FAQs SOLVED BY TAs**

* In Task 3, by "transformation" of categorical values, do you mean getting dummies for all of them? This does'’ play well for bar charts. ***Good idea.***
 
* Task two asks for a unique index for the final data frame. Is it sufficient to give the data frame a numerical index from 0 - > X? Or should the index be made up of a combination of the different attributes? ***Yes, it has to be a meaningful index.***

* How is the “cabin floor” defined? (see homework 1, task 3.3) If cabin letter indicates the floor, what about the guy in cabin “F E46”? Also the “T” seems like a typo. There were 5 such cases..And what about people who reserved more cabins. **Argue about your decisions; Real world datasets are dirty! (Useful: https://www.encyclopedia-titanica.org/titanic-deckplans)**

# Imports

In [ ]:
import math
import pandas as pd
import numpy as np
import glob
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from calendar import monthrange
pd.options.mode.chained_assignment = None

%matplotlib inline

# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### 1.1 Constants

In [ ]:
DEFAULT_NAME_COLUMN_DATE = 'Date'
DEFAULT_NAME_COLUMN_DESCRIPTION = 'Description'
DEFAULT_NAME_COLUMN_TOTAL = 'Totals'
DEFAULT_NAME_COLUMN_COUNTRY = 'Country'

DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES = 'New cases'
DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS = 'New deaths'

DEFAULT_DATA_FORMAT = '%Y-%m-%d'

The definitions of the basic schema and formats for the given dataset are stored in the following dictionaries

In [ ]:
country_keys = ['guinea', 'liberia', 'sl']
paths = {x: DATA_FOLDER + '/ebola/'+ x + '_data' for x in country_keys}
countries = {'guinea': 'Guinea', 'liberia': 'Liberia', 'sl': 'Sierra Leone'}

wanted_columns = {'guinea': ['Date', 'Description', 'Totals'],
                      'liberia': ['Date', 'Variable', 'National'],
                      'sl': ['date', 'variable', 'National']}

date_original_formats = {'guinea': ['%Y-%m-%d', '%y-%m-%d'],
                         'liberia': ['%m/%d/%Y', '%m/%d/%y'],
                         'sl': ['%Y-%m-%d', '%y-%m-%d']}

### 1.2 Auxiliary functions

In [ ]:
def sanitize_date(date, original_formats):
    '''
    Transform a date specified in a string with a certain original
    format into a string with the date in the default format.
    '''
    date_rep = None
    try:
        date_rep = datetime.datetime.strptime(date, original_formats[0])
    except ValueError:
        date_rep = datetime.datetime.strptime(date, original_formats[1])
    return date_rep.strftime(DEFAULT_DATA_FORMAT)

In [ ]:
def estimate_daily_from_cum(np_list):
    '''
    Given a list of accumulated data, calculate the data for each day.
    It assumes ordered data.
    '''
    last_known_n = -1
    final_list = list()
    for i, elem in np_list.iteritems():
        #elem = float(elem)
        if (math.isnan(elem)):
            final_list.append(0)
        else:
            if (last_known_n == -1):
                final_list.append(0)
            else:
                final_list.append(elem - last_known_n)
            last_known_n = elem
    return np.array(final_list)

### 1.3 Read files

We create a dictionary `dataframe_list` that contains, for each country, the corresponding dataframe.

After reading each data frame we apply the function `sanitize_date` to the `Date` column to make sure that the format used for the date is consistent, and we then sort the rows in each dataframe according to the `Date`.

In [ ]:
dataframe_list = {}

for i in country_keys:
    temp_list = []
    all_files = glob.glob(paths[i] + '/*.csv')
    for file in all_files:
        temp_df = pd.read_csv(file)
        temp_list.append(temp_df)
    dataframe_list[i] = pd.concat(temp_list)
    
    # Format dates and order dataframes by date
    dataframe_list[i][wanted_columns[i][0]] = np.array([sanitize_date(x, date_original_formats[i])
                                                        for x in dataframe_list[i][wanted_columns[i][0]]])
    
    dataframe_list[i][wanted_columns[i][0]] = pd.to_datetime(dataframe_list[i][wanted_columns[i][0]])
    dataframe_list[i].sort_values(by=wanted_columns[i][0])

### 1.4 Parsing data for each country

The goal of the parsing processes is to obtain a *standard* datataset for each country. We want to extract the useful data for our task while reorganizing the schema and data format to develop a general coherence.

In particular, we want to obtain the daily number of deaths and new cases registered for each country.

***N.B.*** The resulting dataframes do not have the indexing reformatted. Since those are temporary results that will later be concatenated in a single dataframe, we decided to handle the indexing after this operation.

--

#### 1.4.1 Guinea

In order to parse the Guinea dataset we considered the variables ```New deaths registered``` (or ```New deaths registered today``` when it was used instead of the previous one) and ```Total new cases registered so far```. These variables represent the daily number of deaths and cases respectively.

In [ ]:
# Select certain column from the CSV formatted files
parsed_guinea_df = dataframe_list['guinea'][wanted_columns['guinea']]
guinea_wanted_row_list = ['Total new cases registered so far',
                          'New deaths registered',
                          'New deaths registered today']

# Select all the rows that match desired names for 'Description' column
parsed_guinea_df = parsed_guinea_df[parsed_guinea_df['Description'].isin(guinea_wanted_row_list)]

# Rename every row value for 'Description' column 
parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                     guinea_wanted_row_list[0], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES

parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                     guinea_wanted_row_list[1], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS

parsed_guinea_df.loc[parsed_guinea_df['Description'] ==
                     guinea_wanted_row_list[2], 'Description'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS

# Add 'Country' column to dataframe
parsed_guinea_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['guinea']

Sample:

In [ ]:
parsed_guinea_df.head()

#### 1.4.2 Liberia

In order to parse the Liberia dataset we considered the ```Newly reported deaths``` variable, which represents the daily number of deaths. To handle the new cases data we, on the other hand, had to do some manipulations. 

The number of new (daily) cases comes in three different variables (```New Case/s (Suspected)```, ```New Case/s (Probable)``` and ```New Case/s (confirmed)```) that have to be summed up to obtain the total value of new cases.

In [ ]:
# Select certain columns from the CSV formatted files
liberia_df = dataframe_list['liberia'][wanted_columns['liberia']]

liberia_wanted_row_list = ['New Case/s (Suspected)',
                   'New Case/s (Probable)',
                   'New case/s (confirmed)',
                   'Newly reported deaths',
                   'Total suspected cases',
                   'Total probable cases',
                   'Total confirmed cases']

However, the data comes with strangely big new cases values for the last days of the report:

In [ ]:
liberia_df[(liberia_df['Variable'] == 'New case/s (confirmed)') &
           (liberia_df['Date'].apply(lambda x: x.month) == 12)].sort_values(by=['Date'])

We concluded that those numbers couldn't be correct. As we found a correlation between those values and the total new cases accumulator for the past days, we assumed the data as been inputted on the wrong part of the table.

The new cases data for the last month of the report were therefore calculated from the daily difference of the cases accumulators, while we took the daily values for the rest of the months.

In [ ]:
# Create temporary dataframes for new case entries
# Take accumulated cases for the last month of the report
liberia_df_cum_cases_1 = liberia_df[liberia_df['Variable'].isin(liberia_wanted_row_list[4:7])]
liberia_df_cum_cases_1 = liberia_df_cum_cases_1[liberia_df_cum_cases_1['Date'].map(
    lambda x: x.month == 12 and x.day <= 3)]
liberia_df_cum_cases_1 = liberia_df_cum_cases_1.groupby('Date').sum()

liberia_df_cum_cases_2 = liberia_df[liberia_df['Variable'].isin(liberia_wanted_row_list[0:3])]
liberia_df_cum_cases_2 = liberia_df_cum_cases_2[liberia_df_cum_cases_2['Date'].map(
    lambda x: x.month == 12 and x.day > 3)]
liberia_df_cum_cases_2 = liberia_df_cum_cases_2.groupby('Date').sum()

liberia_df_cum_cases = pd.concat([liberia_df_cum_cases_1, liberia_df_cum_cases_2])
liberia_df_cum_cases['National'] = estimate_daily_from_cum(liberia_df_cum_cases['National'])

# Take daily cases for the other months
liberia_df_new_cases = liberia_df[liberia_df['Variable'].isin(liberia_wanted_row_list[0:3])]
liberia_df_new_cases = liberia_df_new_cases[liberia_df_new_cases['Date'].map(
    lambda x: x.month != 12)]

# Get dataframe with new cases for all months
liberia_df_new_cases = pd.concat([liberia_df_new_cases, liberia_df_cum_cases])

# Sum all of the values for 'Probable', 'Variable' and 'Confirmed' new cases
liberia_df_new_cases = liberia_df_new_cases.groupby('Date').sum()

liberia_df_new_cases['Date'] = liberia_df_new_cases.index
liberia_df_new_cases['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES

In [ ]:
# Create temporary dataframe for new deaths entries
liberia_df_new_deaths = liberia_df[liberia_df['Variable'] == liberia_wanted_row_list[3]]
liberia_df_new_deaths['Variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS

In [ ]:
# Create dataframe with new deaths and cases (concatenating the two temporary)
parsed_liberia_df = pd.concat([liberia_df_new_cases, liberia_df_new_deaths])

parsed_liberia_df.rename(columns={'Date': DEFAULT_NAME_COLUMN_DATE,
                                  'Variable': DEFAULT_NAME_COLUMN_DESCRIPTION,
                                  'National': DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)

# Add 'Country' column to dataframe
parsed_liberia_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['liberia']

Sample:

In [ ]:
parsed_liberia_df.head()

#### 1.4.3 Sierra Leone

In order to parse the Sierra Leone dataset we followed similar steps as for Liberia. The new cases value has been computed as the sum of the corresponding variables of the table (```new_noncase```, ```new_suspected```, ```new_probable``` and ```new_confirmed```). 

Since the dataset only had total accumulators for the number of deaths, the new death variable was calculated as the difference between days, as previously done with Liberia. The possible missing data for this operation has been handled by setting them to zero, while keeping the the same delta for known points.

In [ ]:
# Select certain columns from the CSV formatted files
parsed_sierra_df = dataframe_list['sl'][wanted_columns['sl']]

sierra_wanted_row_list = ['new_noncase', 'new_suspected', 'new_probable', 'new_confirmed',
                   'death_confirmed', 'death_probable', 'death_suspected']

In [ ]:
# Create temporary dataframe for new case entries
# (with every row that matches the desired 'variables' column value)
sierra_df_new_cases = parsed_sierra_df[parsed_sierra_df['variable'].isin(
    sierra_wanted_row_list[0:4])]

# Sum all of the values for 'Probable', 'Variable', 'Confirmed' and 'Noncase' new cases
sierra_df_new_cases = sierra_df_new_cases.groupby('date')['National'].apply(
    lambda x: np.array([float(y) for y in x]).sum()).to_frame()

sierra_df_new_cases['date'] = sierra_df_new_cases.index
sierra_df_new_cases['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES

In [ ]:
# Create temporary dataframe for new deaths' accumulator entries
sierra_df_new_deaths_cum = parsed_sierra_df[parsed_sierra_df['variable'].isin(
    sierra_wanted_row_list[4:7])]

# Sum all of the values for 'Probable', 'Suspected' and 'Confirmed' new death accumulators
sierra_df_new_deaths_cum = sierra_df_new_deaths_cum.groupby('date')['National'].apply(
    lambda x: np.array([float(y) for y in x]).sum()).to_frame()

sierra_df_new_deaths_cum['date'] = sierra_df_new_deaths_cum.index
sierra_df_new_deaths_cum['variable'] = 'New deaths accumulator'

# 
sierra_df_new_deaths_cum['National'] = estimate_daily_from_cum(sierra_df_new_deaths_cum['National'])
sierra_df_new_deaths_cum['variable'] = DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS

In [ ]:
parsed_sierra_df = pd.concat([sierra_df_new_cases, sierra_df_new_deaths_cum])

parsed_sierra_df.rename(columns={'date': DEFAULT_NAME_COLUMN_DATE,
                                 'variable': DEFAULT_NAME_COLUMN_DESCRIPTION,
                                 'National': DEFAULT_NAME_COLUMN_TOTAL}, inplace=True)

# Add 'Country' column to dataframe
parsed_sierra_df[DEFAULT_NAME_COLUMN_COUNTRY] = countries['sl']

Sample:

In [ ]:
parsed_sierra_df.head()

### 1.5 Joining the parsed country datatset into a single one

In [ ]:
complete_df = pd.concat([parsed_guinea_df, parsed_liberia_df, parsed_sierra_df])
complete_df.reset_index(inplace=True, drop=True)

Samples:

In [ ]:
complete_df[42:47]

Even after handling the NaN troubles during the parsing for some aspects, we still have some to manage

In [ ]:
complete_df[complete_df[DEFAULT_NAME_COLUMN_TOTAL].isnull()].head()

As for now, we assume that the missing values equal 0

In [ ]:
# Assume NaN values are 0
complete_df = complete_df.fillna(0)

We have seen that the difference in the number of deaths for some rows is negative. It is not a reasonable value and is probably due to an error when keeping track of the accumulated number of deaths, especially during the change of month. We therefore decided to consider 0 instead.

In [ ]:
complete_df[complete_df['Totals'].apply(lambda x: int(x)) < 0]

In [ ]:
complete_df.loc[complete_df['Totals'].apply(lambda x: int(x)) < 0, 'Totals'] = 0

### 1.6 Calculate for each country, the daily average per month of new cases and deaths

We start grouping by ```Country```, ```Description``` and month, and then calculate the average over the total number of days in the month.

In [ ]:
grouped = complete_df.groupby(by=[complete_df.Country, complete_df.Description,
                                  [x.month for x in complete_df.Date]])

In [ ]:
calculated_average_df = pd.DataFrame()

for ((country, description, month), values) in grouped:
    days_in_month = monthrange(2014, month)[1]
    entry = {DEFAULT_NAME_COLUMN_COUNTRY: country, 
             'Description': description, 
             'Month': month, 
             'Average': values[DEFAULT_NAME_COLUMN_TOTAL].apply(lambda x: float(x) / days_in_month).sum()}
    calculated_average_df = calculated_average_df.append([entry])

In [ ]:
death_average_df = calculated_average_df[calculated_average_df[DEFAULT_NAME_COLUMN_DESCRIPTION] ==
                                         DEFAULT_NAME_ROW_DESCRIPTION_NEW_DEATHS]
cases_average_df = calculated_average_df[calculated_average_df[DEFAULT_NAME_COLUMN_DESCRIPTION] ==
                                         DEFAULT_NAME_ROW_DESCRIPTION_NEW_CASES]

calculated_average_df = pd.merge(death_average_df, cases_average_df, on=[DEFAULT_NAME_COLUMN_COUNTRY, 'Month'])
calculated_average_df.drop([DEFAULT_NAME_COLUMN_DESCRIPTION+'_x',
                            DEFAULT_NAME_COLUMN_DESCRIPTION+'_y'], axis=1, inplace=True)

calculated_average_df = calculated_average_df.rename(columns={'Average_x': 'Death monthly average',
                                                              'Average_y': 'Case monthly average'})

After reordering the columns, the final result is shown in the following table. For each country we calculated the average for the new deaths entries as well as the new cases entries

In [ ]:
calculated_average_df = calculated_average_df[['Country', 'Month',
                                               'Death monthly average', 'Case monthly average']]
calculated_average_df

### 1.7 Plots

We plotted the results for a final visualisation of the obtained data. First by country alone in the following bar charts

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 7))

for i, (title, group) in enumerate(calculated_average_df.groupby(DEFAULT_NAME_COLUMN_COUNTRY)):
    group.plot.bar(x='Month', title=title, ax=axes.flat[i])

And lastly as an unique bar chart for all the results in one plot

In [ ]:
calculated_average_df.plot.bar(x=['Month',DEFAULT_NAME_COLUMN_COUNTRY], figsize=(20,7))

# Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

### 2.1 Read files

We started by importing the nine tables in one single big dataframe

In [ ]:
all_files = glob.glob(DATA_FOLDER + '/microbiome' + '/MID*.xls')
raw_df = pd.DataFrame()

for file in all_files:
    file_name = file[file.rfind('/')+1:file.rfind('.')]
    temp_df = pd.read_excel(file, header=None)
    temp_df['BARCODE'] = file_name
    raw_df = raw_df.append(temp_df)

raw_df.head()

We the imported the metadata table in another dataframe

In [ ]:
metadata_df = pd.read_excel(DATA_FOLDER+'/microbiome'+'/metadata.xls')
metadata_df

### 2.2 Merging the dataframes

The final goal is to obtain a single dataframe for the given data. We will need to obtain a table of the following format:

| Description | Group | Tissue | Stool | Other |
|-------------|-------|--------|-------|-------|
|             |       |        |       |       |
|             |       |        |       |       |
|             |       |        |       |       |


We will fill-in the last three collumns with the sample date we have in the given datatframes. The column will be chosen according to what the metadata provides. The same idea will be followed to fill-in the Group collumn with the correct information

We create three dataframe containing the metadata informations for a single Sample type

In [ ]:
stool_metadata_df = metadata_df[metadata_df['SAMPLE'] == 'stool']
tissue_metadata_df = metadata_df[metadata_df['SAMPLE'] == 'tissue']
na_metadata_df = metadata_df[metadata_df['SAMPLE'].isnull()]

Sample:

In [ ]:
tissue_metadata_df

We create three dataframes for the different samples according to what we have in the metatada dataframes obtained before

In [ ]:
raw_df = pd.merge(raw_df, metadata_df[['BARCODE', 'GROUP']], on=['BARCODE'])
raw_df.rename(columns={0: 'Description', 1: 'Sample'}, inplace=True)

stool_df = raw_df[raw_df['BARCODE'].isin(list(stool_metadata_df['BARCODE']))].drop('BARCODE', axis=1)
stool_df.rename(columns={'Sample': 'Stool'}, inplace=True)
              
tissue_df = raw_df[raw_df['BARCODE'].isin(list(tissue_metadata_df['BARCODE']))].drop('BARCODE', axis=1)
tissue_df.rename(columns={'Sample': 'Tissue'}, inplace=True)
              
na_df = raw_df[raw_df['BARCODE'].isin(list(na_metadata_df['BARCODE']))].drop('BARCODE', axis=1)
na_df.rename(columns={'Sample': 'Other'}, inplace=True)

Sample:

In [ ]:
stool_df.head()

We merge those three temporary dataframes into a single one

In [ ]:
merged_df = pd.merge(stool_df, tissue_df, how='outer', on=['Description', 'GROUP'])
merged_df = pd.merge(merged_df, na_df, how='outer', on=['Description', 'GROUP'])
merged_df.head()

To complete the task, we fill the NaN values of the dataset with the 'unknown' value and we clear the general format of the table

In [ ]:
# Fill of the 'unknown' value for NaNs
merged_df.fillna('unknown', inplace=True)

# Final cleaning up of the table schema
merged_df.rename(columns={'GROUP':'Group'}, inplace=True)

merged_df.head()

In [ ]:
## OTHER POSSIBILITY FOR THIS TASK

def return_from_barcode(barcode, dictionary):
    for x in dictionary:
        if barcode in dictionary[x]:
            return x
    return None

parsed_df = raw_df.copy()

groups_dictionary = {}
for group in metadata_df['GROUP'].unique():
    groups_dictionary[group] = metadata_df[metadata_df['GROUP'] == group]['BARCODE'].values

parsed_df['GROUP'] = parsed_df['BARCODE'].map(lambda x: return_from_barcode(x, groups_dictionary))
parsed_df.rename(columns={1: 'VALUE', 0: 'GENUS'}, inplace=True)
parsed_df.reset_index(inplace=True, drop=True)

samples_dictionary = {'Nan': ['MID1']}
for sample in metadata_df['SAMPLE'].unique():
    samples_dictionary[sample] = metadata_df[metadata_df['SAMPLE'] == sample]['BARCODE'].values

parsed_df['SAMPLE'] = parsed_df['BARCODE'].map(lambda x: return_from_barcode(x, samples_dictionary))

parsed_df.head()

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
titanic_df = pd.read_excel(DATA_FOLDER+'/titanic.xls')
titanic_df

In [ ]:
metadata_df = pd.read_html(DATA_FOLDER+'/titanic.html', header=0)

# Fills the null values in the first column with the last valid value
metadata_df[1]['Variable'].fillna(method='ffill', inplace=True)

In [ ]:
metadata_df[0]

In [ ]:
metadata_df[1]

### 3.1 Description of attributes
Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`.

In [ ]:
for attribute in metadata_df[0]['Name']:
    if titanic_df[attribute].isnull().any():
        print(attribute+' has null values')

* pclass: integer; 3 levels: 1, 2 or 3; Can be turned into an integer.
* survived: binary, 0 means that the passenger survived and 1 means that did not survive. Equivalent to a categorical attribute with 2 levels
* name: string
* sex: string; 2 levels: male or female
* age: real number; If it's less than 1 then the passenger was younger than 1 year old. If it's in the form xx.5 then the age is an estimation; (null values possible)
* sibsp: integer
* parch: integer
* ticket: string, but could be reformatted to be an integer
* fare: real number (null values possible)
* cabin: string; 187 levels (null values possible)
* embarked: string; 3 levels: S, C or Q (null values possible)
* boat: string; 28 levels (null values possible)
* body: integer (null values possible)
* home.dest: string (null values possible)

The ones that can be categorical are:

* pclass
* survived
* sex
* cabin
* embarked
* boat

All of them appear in `metadata_df[1]` except `survived`:

In [ ]:
value_range_categorical = {'survived': titanic_df['survived'].unique()}
for attribute in metadata_df[1]['Variable'].unique():
    value_range_categorical[attribute] = np.array(metadata_df[1][metadata_df[1]['Variable'] == attribute]['Levels'])

In [ ]:
# TRANSFORMATION OPTION 2
# survived is already binary
# We don't transform 'boat' because it won't be used
# We create a variable 'cabin_floor' and then create dummy variables for it

titanic_df_2 = titanic_df.copy()

In [ ]:
titanic_df_2['cabin'].fillna(value='', inplace=True)

In [ ]:
def find_floor(cabin):
    # Cabin T is in the boat deck
    # We assume cabins in the format F Gxx are in deck F (and therefore in floor 6)
    map_dictionary = {'T': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    if not cabin:
        return None
    else:
        cabin_list = str(cabin).split()
        # Assumes the last cabin listed is the valid one
        # This also makes sure that F Gxx counts as a cabin in deck F (assumption)
        valid_cabin = cabin_list[0]
        return map_dictionary[valid_cabin[0]]

In [ ]:
titanic_df_2['cabin_floor'] = titanic_df_2['cabin'].map(lambda x: find_floor(x))

In [ ]:
pd.get_dummies(titanic_df_2, columns=['pclass', 'sex', 'embarked','cabin_floor'])

### 3.2 Histogram plots

We will provide four histograms in the following section. Those plots visualize different amount of passengers from the dataset according to the `travel class`, `sex`, `embarked port` and `age`.

Axuliary function for the parsing of the dataset to plot:

In [ ]:
def get_ord_from_str(string):
    ord_dic = {'nan':0, 'S':1, 'Q':2, 'C':3,}
    stripped_string = str(string).strip().strip('\n')
    return ord_dic.get(stripped_string, -1)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

# Travel class dataset and histogram setup
travel_class_df = pd.DataFrame(titanic_df[['pclass']])
travel_class_df.plot.hist(ax=axes[0,0])

plt.sca(axes[0, 0])
plt.title('Passengers by class')
plt.legend(loc='upper center')
plt.sca(axes[0, 0])
plt.xticks(np.arange(1.0, 4.0, 1.0), ('1st class', '2nd class', '3rd class'))


# Sex dataset and histogram setup
sex_df = pd.DataFrame(titanic_df[['sex']])
sex_df['sex'] = sex_df['sex'].apply(lambda x : 1 if x == 'male' else 0) #mapping 
sex_df.plot.hist(ax=axes[0,1])

plt.sca(axes[0, 1])
plt.title('Passengers by sex')
plt.legend(loc='upper center')
plt.xticks(np.arange(0.0, 2.0, 1.0), ('female', 'male'))


# Embarked port dataset and histogram setup
embarked_df = pd.DataFrame(titanic_df[['embarked']])
embarked_df = embarked_df.apply(lambda x: x.apply(lambda y: get_ord_from_str(y)))
embarked_df.plot.hist(ax=axes[1,0])

plt.sca(axes[1, 0])
plt.title('Passengers by port')
plt.legend(loc='upper right')
plt.xticks(np.arange(0.0, 4.0, 1.0), ('NaN', 'Southampton', 'Queenstown', 'Cherbourg'))


# Age dataset and histogram setup
age_df = pd.DataFrame(titanic_df[['age']])
age_df.plot.hist(ax=axes[1,1])

plt.sca(axes[1, 1])
plt.title('Passengers by age')
plt.legend(loc='upper center')
plt.xticks(np.arange(0.0, 100.0, 10.0))


plt.show()

### 3.3 Passengers by cabin floor
Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.

ax = sns.countplot(x='cabin_floor', data=titanic_df_2)

In [ ]:
passengers_per_cabin_floor = {}
for floor in titanic_df_2['cabin_floor'].unique():
    if str(floor) != 'nan':
        passengers_per_cabin_floor[int(floor)] = titanic_df_2[titanic_df_2['cabin_floor'] == floor]['cabin_floor'].count()

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.pie(list(passengers_per_cabin_floor.values()), labels=passengers_per_cabin_floor.keys(), autopct='%1.0f%%')
plt.axis('equal')
plt.title('Passengers per floor')
plt.show()

### 3.4 Survivors per class
For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.

In [ ]:
survivors_per_class_df = titanic_df[['pclass', 'survived']].groupby(['pclass', 'survived']).agg(len)

# Plot a pie chart for each class
for pclass in range(1,4):
    fig = plt.figure(figsize=(5,5))
    plt.pie(survivors_per_class_df[pclass], labels=['Died', 'Survived'], autopct='%1.0f%%')
    plt.axis('equal')
    plt.title('Class '+str(pclass))
    plt.show()

### 3.5 Survivors per class and sex

In [ ]:
# Dataset for survived entries on their class and sex atributes
survivors_per_class_sex_df = pd.DataFrame(titanic_df[titanic_df.survived == 1][['pclass', 'sex']])

# Split of the data in two datasets (one for male, one for female)
male_df = survivors_per_class_sex_df[survivors_per_class_sex_df['sex'] == 'male']
female_df = survivors_per_class_sex_df[survivors_per_class_sex_df['sex'] != 'male']

# Plotting of the two datasets as stacked histogram
fig = plt.figure(figsize=(15,7))
plt.hist([male_df['pclass'], female_df['pclass']], label=['male', 'female'], stacked=True)
plt.title('Survivors by class')
plt.legend(loc='upper center')
plt.xticks(np.arange(1.0, 4.0, 1.0))
plt.show()

### 3.6 Age categories

We decided to define the `age category` in the following way: we took the median age value and then grouped all the people younger in category 1 and all the older ones in category 2

In [ ]:
# Define the age category
titanic_df['age_category'] = titanic_df['age'].map(lambda x: 1 if x <= titanic_df['age'].median() 
                                                                   else 2 if x > titanic_df['age'].median() 
                                                                             else None)

age_categories_df = pd.DataFrame(titanic_df[titanic_df.survived == 1][['age_category', 'pclass', 'sex']]
                                 .groupby(['age_category', 'pclass', 'sex'])
                                 .agg(len))
# Compute the proportion ratio
age_categories_df = age_categories_df.apply(lambda x : x / age_categories_df.values.sum())

# Re indexing of the table
age_categories_df.reset_index(inplace=True)

# Renaming of the datatset schema
age_categories_df.rename(columns={'age_category':'age category', 'pclass':'class', 0:'proportion'}, inplace=True)

age_categories_df